In [1]:
import torch
import torch.nn as nn

In [50]:
x = torch.randn(1,5,5)
x.shape, x

(torch.Size([1, 5, 5]),
 tensor([[[ 1.4518,  0.4856, -0.5389,  0.2733, -0.3157],
          [ 0.3609, -1.0716, -0.8421, -0.0280,  1.6671],
          [-0.1299, -0.9957,  0.2819,  0.1756, -1.1801],
          [ 1.1808, -0.9658, -1.1960,  1.0513, -0.0851],
          [-0.1109,  0.4165,  3.0197,  1.5542,  0.2885]]]))

Let's create a simple convolutional network with 1 in_channel & 1 out_channel. The kernel size = 3.

For illustrative purposes, we set all $3 \times 3 = 9$ kernel values to 1. So basically, the network sums all $3 \times 3$ and outputs them. So the network output is smaller than the input!

In [74]:
m = nn.Conv2d(in_channels = 1,out_channels = 1,kernel_size = 3)
m.weight = torch.nn.Parameter(torch.ones_like(m.weight))
m.state_dict()

OrderedDict([('weight',
              tensor([[[[1., 1., 1.],
                        [1., 1., 1.],
                        [1., 1., 1.]]]])),
             ('bias', tensor([-0.0658]))])

Calculate the model output

In [75]:
m(x)

tensor([[[-1.0637, -2.3257, -0.5727],
         [-3.4432, -3.6562, -0.2212],
         [ 1.4349,  3.2759,  3.8442]]], grad_fn=<SqueezeBackward1>)

Take a $3 \times 3$ slice of the input

In [76]:
x[0,0:3,0:3]

tensor([[ 1.4518,  0.4856, -0.5389],
        [ 0.3609, -1.0716, -0.8421],
        [-0.1299, -0.9957,  0.2819]])

Now do the sum and bias addition "by hand" and we see that this matches the `(0,0)` position in `m(x)`

In [77]:
torch.sum(x[0,0:3,0:3]) + m.bias

tensor([-1.0637], grad_fn=<AddBackward0>)

Now the case with general weights

In [78]:
m = nn.Conv2d(in_channels = 1,out_channels = 1,kernel_size = 3)
m.state_dict()

OrderedDict([('weight',
              tensor([[[[ 0.1734,  0.2302,  0.2847],
                        [ 0.2793, -0.2692,  0.2193],
                        [-0.2059,  0.1034,  0.1096]]]])),
             ('bias', tensor([0.0002]))])

In [79]:
m(x)

tensor([[[ 0.3697,  0.2129, -0.1515],
         [-0.6041, -0.5124,  0.4407],
         [ 0.5533,  0.6222, -1.3116]]], grad_fn=<SqueezeBackward1>)

Piecewise multiplication, then sum & add the bias

In [80]:
torch.sum(m.weight * x[0,0:3,0:3]) + m.bias

tensor([0.3697], grad_fn=<AddBackward0>)

In [81]:
torch.sum(m.weight * x[0,1:4,0:3]) + m.bias

tensor([-0.6041], grad_fn=<AddBackward0>)

In [82]:
torch.sum(m.weight * x[0,0:3,1:4]) + m.bias

tensor([0.2129], grad_fn=<AddBackward0>)

Different stride

In [85]:
m = nn.Conv2d(in_channels = 1,out_channels = 1,kernel_size = 3, stride = 2)
m.state_dict()

OrderedDict([('weight',
              tensor([[[[ 0.2815, -0.0375, -0.1130],
                        [-0.2259, -0.1822,  0.2419],
                        [-0.0840, -0.2244, -0.0424]]]])),
             ('bias', tensor([0.0358]))])

In [86]:
m(x)

tensor([[[ 0.6196,  0.4951],
         [-0.5877, -0.3147]]], grad_fn=<SqueezeBackward1>)

In [87]:
torch.sum(m.weight * x[0,0:3,0:3]) + m.bias

tensor([0.6196], grad_fn=<AddBackward0>)

In [88]:
torch.sum(m.weight * x[0,2:5,0:3]) + m.bias

tensor([-0.5877], grad_fn=<AddBackward0>)